# NOTEBOOK PARA AGREGAR LA COLUMNA DE TIEMPO DEL SELP

Referencias:

* https://stackoverflow.com/questions/17098654/how-to-store-a-dataframe-using-pandas
* https://datascience.stackexchange.com/questions/33053/how-do-i-compare-columns-in-different-data-frames

In [1]:
# Este notebook es para agregar la columna de tiempo del selected pulse con sus respectivos raw_hits
import pandas as pd
import os

csv_path = "F:\DATA_TFM\CSVs\\"
csvarr = os.listdir(csv_path); csvarr = sorted(csvarr)

In [4]:
###############################
#### FUNCIONES IMPORTANTES ####
###############################

# Funcion para obtener los indices de un archivo
def indices(df):
    ind_startev = [];ind_runid = [];ind_weights = [];ind_muon = []
    ind_bbfit = [];ind_bb_selp = [];ind_endev = [];#ind_aafit = []
    for i in range(data.shape[0]):
        tempstr = data[0][i]
        if "start_event" in tempstr: ind_startev.append(i)
        if "UTC" in tempstr: ind_runid.append(i)
        if "weights" in tempstr: ind_weights.append(i)
        if "muon" in tempstr: ind_muon.append(i)
        #if "aafit" in tempstr:  ##NO LO CALCULARE!!! OBTENDRE LOS VALORES DE OTRA FORMA
        #    ind_aafit.append(i) ##COMO TENGO EL INDICE DE BBFIT, RESTO 1 LINEA Y ASIGNO VALORES
        if "bbfit" in tempstr:   ##SEGUN LA EXISTENCIA O NO DE LA LINEA!!!! EUREKA!!!
            ind_bbfit.append(i)
        if "selected" in tempstr: ind_bb_selp.append(i)
        if "end_event" in tempstr: ind_endev.append(i)
    return(ind_startev,ind_runid,ind_weights,ind_muon,ind_bbfit,ind_bb_selp,ind_endev)

#Función para limpiar un hit (lista) de elementos tipo dobles espacios y simples
def limpiahit(templist):
    cleanlist = []
    for j in range(len(templist)):
        if templist[j]!="":
            cleanlist.append(templist[j])
    return(cleanlist)

#Función para buscar los "rows" que coinciden en línea y piso de solo un selected pulse
def func_lineas_coinc(linea,piso,selp,hits):
    lineas_coinc = []
    for i in range(len(hits)):
        if hits[i][2]==linea and hits[i][3]==piso:
            lineas_coinc.append(hits_raw_limpios[i])
    return(lineas_coinc)

#Función para obtener todas las líneas de los selected pulse de un evento
def func_lineas_evento(sel_hits_limpios,hits_raw_limpios):
    lineas_selp_evento = []
    for i in range(len(sel_hits_limpios)):
        lindet = sel_hits_limpios[i][2] #Linea detector a buscar
        storey = sel_hits_limpios[i][3] #Piso detector a buscar
        lineas = func_lineas_coinc(lindet,storey,sel_hits_limpios[i],hits_raw_limpios)
        t_selp = sel_hits_limpios[i][-3]
        #El siguiente bucle añade el tiempo del selected pulse que le corresponde a las lineas seleccionadas
        #AUNQUE PARECE SENCILLO EL CÓDIGO, PENSAR LA SOLUCIÓN SI QUE ME LLEVÓ UN BUEN RATO
        for k in range(len(lineas)):
            #print("Antes de añadir el tiempo\n",lineas[k])
            lineas[k].append(t_selp)
            #print("Despues de añadir el tiempo\n",lineas[k])
        #print(lineas,"\n",sel_hits_limpios[i][-3])
        #Debo añadir las lineas de cada selected pulse a otra lista como "renglones"
        for j in range(len(lineas)):
            lineas_selp_evento.append(lineas[j])
    return(lineas_selp_evento)

#Función para verificar si los indices tienen el mismo # de elementos
def longitudes(i1,i2,i3,i4,i5,i6,i7):
    a=len(i1);b=len(i2);c=len(i3);d=len(i4);e=len(i5);f=len(i6);g=len(i7)
    #print(a,b,c,d,e,f,g)
    if (a==b and b==c and c==d and d==e and e==f and f==g): return(True)
    else: return(False)
    return()

#Función que hice para buscar un frameID en específico en el dataframe final
def func_frameID(num,df):
    tempdf = df.loc[df['frameID'].isin([str(num)])]
    #print(tempdf.head(5))
    return(tempdf)

In [5]:
###### CÓDIGO PARA RECORRER TODOS LOS ARCHIVOS Y OBTENER LA COLUMNA DEL TIEMPO DE LOS SELECTED PULSES ######

tiempo_selp = [] #Lista que se convertirá en la columna a agregar al dataframe/archivo que ya tengo y que le falto esta.

best_hits = [] #Definimos la lista donde se guardarán los hits seleccionados

#Primer bucle donde se leerá la data de cada archivo
for z in range(len(csvarr)): #Aqui le deje el len porque estoy trabajando en casa con la laptop
    #Abrimos el archivo, de acuerdo con el índice z
    file = csv_path+csvarr[z]
    data = pd.read_csv(file, header=None)
    #Obteniendo los indices necesarios
    ind_startev,ind_runid,ind_weights,ind_muon,ind_bbfit,ind_bb_selp,ind_endev = indices(data)
    print("El archivo trabajando es:",file,"eventos:",len(ind_startev))
    #Obtenemos la metadata runID e interactionID ya que es la misma para cada archivo
    runID = int(list(str(data[0][ind_runid[0]]).split(" "))[0])#Corregí esta línea era 0 en ambos
    interactionID = [lambda:1, lambda:2]["anumu" in str(file)]()
    print("Entrando al bucle para obtener los best-hits")
    bhcount_file = 0; rhcount_file = 0 #Contadores para "monitorear"
    ### Bucle para buscar los best hits en cada evento del archivo
    for j in range(len(ind_startev)):
        #Obtengo el frameID y trigger_counter, que son diferentes para cada evento
        temprunid = list(data[0][ind_runid[j]].split(" "))
        frameID = temprunid[1]; trigger_counter = temprunid [2]
        #Encontramos primero el # de lineas de rawhits y selected pulses
        difrawhits = ind_bb_selp[j] - ind_bbfit[j]
        difselhits = ind_endev[j] - ind_bb_selp[j]
        start_rawhit = ind_bbfit[j]+1;
        start_selhit = ind_bb_selp[j]+1
        raw_hits = []; hits_raw_limpios = [];sel_hits = []; sel_hits_limpios = []      
        #Obteniendo solo los raw_hits de un evento
        for i in range(difrawhits-1):
            temprawhit = list(data[0][start_rawhit+i].split(" "))
            raw_hits.append(temprawhit)
            #Limpiamos los raw hits obtenidos del evento
        for i in range(len(raw_hits)): hits_raw_limpios.append(limpiahit(raw_hits[i]))
        #Obteniendo solo los selected pulses (hits) del mismo evento
        for i in range(difselhits-1):
            tempselhit = list(data[0][start_selhit+i].split(" "))
            sel_hits.append(tempselhit)
        #Limpiamos los raw hits obtenidos del evento
        for i in range(len(sel_hits)): sel_hits_limpios.append(limpiahit(sel_hits[i]))
        #A continuación obtenemos todas las lineas que coinciden con cada selected pulse del evento
        templist = func_lineas_evento(sel_hits_limpios,hits_raw_limpios)
        #Agregamos la metadata que le falta a cada "row"
        #tiempo_selp = limpiahit(data[0][ind_bb_selp[j]+1+i].split(" "))[-3]
        #if lin_det_selp == lin_det_raw_hit && piso_det_selp == piso_det_raw_hit:
        #    tiempo_selp = sel_hits_limpios[0][-3] #Aqui debo seleccionar el tiempo... pero como???? X-(
        for i in range(len(templist)):
            #templist[i].insert(0,tiempo_selp) #Aqui meto el tiempo del selected pulse Debi haber quitado esto :(
            templist[i].insert(0,interactionID)
            templist[i].insert(0,trigger_counter)
            templist[i].insert(0,frameID)
            templist[i].insert(0,runID)
        for k in range(len(templist)):
            best_hits.append(templist[k])
        rhcount_file += len(raw_hits)
        bhcount_file += len(templist)
    print("Raw_hits:",rhcount_file,"Número de best_hits:",bhcount_file)
    #inicio += 1 #Incremento para abrir el archivo correcto

print("\nFin de la extracción de la data")

El archivo trabajando es: F:\DATA_TFM\CSVs\MC_025800_anumu_CC_a_reco.i3.csv eventos: 256
Entrando al bucle para obtener los best-hits
Raw_hits: 41006 Número de best_hits: 10018
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_025800_numu_CC_a_reco.i3.csv eventos: 223
Entrando al bucle para obtener los best-hits
Raw_hits: 35508 Número de best_hits: 8469
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_025880_anumu_CC_a_reco.i3.csv eventos: 271
Entrando al bucle para obtener los best-hits
Raw_hits: 43087 Número de best_hits: 10554
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_025880_numu_CC_a_reco.i3.csv eventos: 230
Entrando al bucle para obtener los best-hits
Raw_hits: 35388 Número de best_hits: 7889
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_025920_anumu_CC_a_reco.i3.csv eventos: 273
Entrando al bucle para obtener los best-hits
Raw_hits: 37399 Número de best_hits: 10225
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_025920_numu_CC_a_reco.i3.csv eventos: 284
Entrando al bucle para obtener 

Raw_hits: 50878 Número de best_hits: 11411
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_026820_numu_CC_a_reco.i3.csv eventos: 229
Entrando al bucle para obtener los best-hits
Raw_hits: 45438 Número de best_hits: 9202
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_026830_anumu_CC_a_reco.i3.csv eventos: 270
Entrando al bucle para obtener los best-hits
Raw_hits: 47189 Número de best_hits: 10832
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_026830_numu_CC_a_reco.i3.csv eventos: 227
Entrando al bucle para obtener los best-hits
Raw_hits: 39397 Número de best_hits: 8889
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_026870_anumu_CC_a_reco.i3.csv eventos: 261
Entrando al bucle para obtener los best-hits
Raw_hits: 38203 Número de best_hits: 9664
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_026870_numu_CC_a_reco.i3.csv eventos: 263
Entrando al bucle para obtener los best-hits
Raw_hits: 38561 Número de best_hits: 9963
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_026920_anumu_CC_a_reco.i3.csv eve

El archivo trabajando es: F:\DATA_TFM\CSVs\MC_027750_anumu_CC_a_reco.i3.csv eventos: 281
Entrando al bucle para obtener los best-hits
Raw_hits: 41741 Número de best_hits: 10477
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_027750_numu_CC_a_reco.i3.csv eventos: 273
Entrando al bucle para obtener los best-hits
Raw_hits: 41962 Número de best_hits: 11265
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_027760_anumu_CC_a_reco.i3.csv eventos: 284
Entrando al bucle para obtener los best-hits
Raw_hits: 41605 Número de best_hits: 10753
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_027760_numu_CC_a_reco.i3.csv eventos: 286
Entrando al bucle para obtener los best-hits
Raw_hits: 41175 Número de best_hits: 10586
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_027830_anumu_CC_a_reco.i3.csv eventos: 273
Entrando al bucle para obtener los best-hits
Raw_hits: 45170 Número de best_hits: 10273
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_027830_numu_CC_a_reco.i3.csv eventos: 253
Entrando al bucle para obtene

Raw_hits: 70977 Número de best_hits: 14527
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_028790_numu_CC_a_reco.i3.csv eventos: 468
Entrando al bucle para obtener los best-hits
Raw_hits: 69672 Número de best_hits: 14596
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_028850_anumu_CC_a_reco.i3.csv eventos: 389
Entrando al bucle para obtener los best-hits
Raw_hits: 63193 Número de best_hits: 12789
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_028850_numu_CC_a_reco.i3.csv eventos: 377
Entrando al bucle para obtener los best-hits
Raw_hits: 61690 Número de best_hits: 12989
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_028860_anumu_CC_a_reco.i3.csv eventos: 405
Entrando al bucle para obtener los best-hits
Raw_hits: 65849 Número de best_hits: 13090
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_028860_numu_CC_a_reco.i3.csv eventos: 357
Entrando al bucle para obtener los best-hits
Raw_hits: 58058 Número de best_hits: 11764
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_028870_anumu_CC_a_reco.i3.csv

Raw_hits: 39680 Número de best_hits: 10339
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_029360_anumu_CC_a_reco.i3.csv eventos: 318
Entrando al bucle para obtener los best-hits
Raw_hits: 41787 Número de best_hits: 10852
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_029360_numu_CC_a_reco.i3.csv eventos: 286
Entrando al bucle para obtener los best-hits
Raw_hits: 38261 Número de best_hits: 9929
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_029370_anumu_CC_a_reco.i3.csv eventos: 288
Entrando al bucle para obtener los best-hits
Raw_hits: 39856 Número de best_hits: 9755
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_029370_numu_CC_a_reco.i3.csv eventos: 286
Entrando al bucle para obtener los best-hits
Raw_hits: 39833 Número de best_hits: 9928
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_029420_anumu_CC_a_reco.i3.csv eventos: 399
Entrando al bucle para obtener los best-hits
Raw_hits: 51183 Número de best_hits: 11789
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_029420_numu_CC_a_reco.i3.csv ev

...

Raw_hits: 181065 Número de best_hits: 16894
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_084290_numu_CC_a_reco.i3.csv eventos: 656
Entrando al bucle para obtener los best-hits
Raw_hits: 169214 Número de best_hits: 17425
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_084320_anumu_CC_a_reco.i3.csv eventos: 2
Entrando al bucle para obtener los best-hits
Raw_hits: 354 Número de best_hits: 26
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_084320_numu_CC_a_reco.i3.csv eventos: 593
Entrando al bucle para obtener los best-hits
Raw_hits: 119916 Número de best_hits: 14391
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_084330_anumu_CC_a_reco.i3.csv eventos: 597
Entrando al bucle para obtener los best-hits
Raw_hits: 168085 Número de best_hits: 15002
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_084330_numu_CC_a_reco.i3.csv eventos: 690
Entrando al bucle para obtener los best-hits
Raw_hits: 196253 Número de best_hits: 18440
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_084340_anumu_CC_a_reco.i3.csv e

El archivo trabajando es: F:\DATA_TFM\CSVs\MC_084570_numu_CC_a_reco.i3.csv eventos: 681
Entrando al bucle para obtener los best-hits
Raw_hits: 136352 Número de best_hits: 16203
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_084580_anumu_CC_a_reco.i3.csv eventos: 711
Entrando al bucle para obtener los best-hits
Raw_hits: 147176 Número de best_hits: 15600
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_084580_numu_CC_a_reco.i3.csv eventos: 702
Entrando al bucle para obtener los best-hits
Raw_hits: 149874 Número de best_hits: 17104
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_084590_anumu_CC_a_reco.i3.csv eventos: 715
Entrando al bucle para obtener los best-hits
Raw_hits: 138800 Número de best_hits: 15753
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_084590_numu_CC_a_reco.i3.csv eventos: 704
Entrando al bucle para obtener los best-hits
Raw_hits: 136363 Número de best_hits: 16191
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_084600_anumu_CC_a_reco.i3.csv eventos: 716
Entrando al bucle para o

Raw_hits: 85802 Número de best_hits: 13485
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_084860_anumu_CC_a_reco.i3.csv eventos: 716
Entrando al bucle para obtener los best-hits
Raw_hits: 111289 Número de best_hits: 14651
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_084860_numu_CC_a_reco.i3.csv eventos: 651
Entrando al bucle para obtener los best-hits
Raw_hits: 101458 Número de best_hits: 13394
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_084890_anumu_CC_a_reco.i3.csv eventos: 737
Entrando al bucle para obtener los best-hits
Raw_hits: 164369 Número de best_hits: 17055
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_084890_numu_CC_a_reco.i3.csv eventos: 650
Entrando al bucle para obtener los best-hits
Raw_hits: 146303 Número de best_hits: 15010
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_084900_anumu_CC_a_reco.i3.csv eventos: 812
Entrando al bucle para obtener los best-hits
Raw_hits: 147949 Número de best_hits: 17488
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_084900_numu_CC_a_reco.i

El archivo trabajando es: F:\DATA_TFM\CSVs\MC_085160_anumu_CC_a_reco.i3.csv eventos: 724
Entrando al bucle para obtener los best-hits
Raw_hits: 198224 Número de best_hits: 18350
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_085160_numu_CC_a_reco.i3.csv eventos: 639
Entrando al bucle para obtener los best-hits
Raw_hits: 174868 Número de best_hits: 15740
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_085170_anumu_CC_a_reco.i3.csv eventos: 811
Entrando al bucle para obtener los best-hits
Raw_hits: 184120 Número de best_hits: 19269
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_085170_numu_CC_a_reco.i3.csv eventos: 767
Entrando al bucle para obtener los best-hits
Raw_hits: 172657 Número de best_hits: 17812
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_085180_anumu_CC_a_reco.i3.csv eventos: 788
Entrando al bucle para obtener los best-hits
Raw_hits: 176625 Número de best_hits: 18473
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_085180_numu_CC_a_reco.i3.csv eventos: 732
Entrando al bucle para o

Raw_hits: 144401 Número de best_hits: 12933
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_085470_anumu_CC_a_reco.i3.csv eventos: 557
Entrando al bucle para obtener los best-hits
Raw_hits: 120874 Número de best_hits: 13965
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_085470_numu_CC_a_reco.i3.csv eventos: 495
Entrando al bucle para obtener los best-hits
Raw_hits: 108155 Número de best_hits: 12639
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_085500_anumu_CC_a_reco.i3.csv eventos: 767
Entrando al bucle para obtener los best-hits
Raw_hits: 136675 Número de best_hits: 15954
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_085500_numu_CC_a_reco.i3.csv eventos: 726
Entrando al bucle para obtener los best-hits
Raw_hits: 130522 Número de best_hits: 16717
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_085510_anumu_CC_a_reco.i3.csv eventos: 692
Entrando al bucle para obtener los best-hits
Raw_hits: 151263 Número de best_hits: 15111
El archivo trabajando es: F:\DATA_TFM\CSVs\MC_085510_numu_CC_a_reco.

In [40]:
#Viendo como quedo la lista best_hits que se debera convertir en df
for i in range(4):
    print(best_hits[i])

[25800, '73557', '0', 2, [], 'hit', '76', '4', '19', '0', '45.532615', '9.219245', '90.040108', '0.693344', '0.138831', '-0.707106', '296.559100', '1.089169', '327.587128', '137.146191']
[25800, '73557', '0', 2, [], 'hit', '77', '4', '19', '0', '45.532615', '9.219245', '90.040108', '0.693344', '0.138831', '-0.707106', '137.146191', '0.879543', '327.587128', '137.146191']
[25800, '73557', '0', 2, [], 'hit', '78', '4', '19', '1', '44.584223', '9.539780', '90.040108', '-0.466903', '0.531039', '-0.707106', '153.189412', '0.772948', '281.085968', '137.146191']
[25800, '73557', '0', 2, [], 'hit', '67', '4', '20', '0', '45.484239', '9.361442', '104.567108', '0.634163', '0.312791', '-0.707106', '168.470446', '1.070132', '82.550049', '167.488297']


In [42]:
#Removiendo las "columnas" que no necesitaremos de la lista best_hits
for i in range(len(best_hits)):
    best_hits[i].remove([])
    best_hits[i].remove('hit')
    
for i in range(4):
    print(best_hits[i])

[25800, '73557', '0', 2, '76', '4', '19', '0', '45.532615', '9.219245', '90.040108', '0.693344', '0.138831', '-0.707106', '296.559100', '1.089169', '327.587128', '137.146191']
[25800, '73557', '0', 2, '77', '4', '19', '0', '45.532615', '9.219245', '90.040108', '0.693344', '0.138831', '-0.707106', '137.146191', '0.879543', '327.587128', '137.146191']
[25800, '73557', '0', 2, '78', '4', '19', '1', '44.584223', '9.539780', '90.040108', '-0.466903', '0.531039', '-0.707106', '153.189412', '0.772948', '281.085968', '137.146191']
[25800, '73557', '0', 2, '67', '4', '20', '0', '45.484239', '9.361442', '104.567108', '0.634163', '0.312791', '-0.707106', '168.470446', '1.070132', '82.550049', '167.488297']


In [47]:
#Escribiendo la lista a disco ya que se acabó la memoria
import csv

with open("F:\DATA_TFM\CLEANDATA\\best_hits_listas_10Jul.csv", "w") as f:
    wr = csv.writer(f, delimiter=" ")
    wr.writerows(best_hits)

## Después de reiniciar el notebook, ejecutare lo siguiente para pasar a dataframe la lista

In [ ]:
columnas = ['runID','frameID','trigger_counter','interactionID','#hit','lin_det','piso_det','mod_opt',
            'X\'','Y\'','Z\'','X','Y','Z','tiempo','amp','frec_hits','t_selp']

cleandata = "F:\DATA_TFM\CLEANDATA\\"
df = pd.read_csv(cleandata+"best_hits_listas_10Jul.csv", header=None, names=columnas, delim_whitespace=True)

In [8]:
print(df.shape)
df.head(5)

(63520301, 18)


,runID,frameID,trigger_counter,interactionID,#hit,lin_det,piso_det,mod_opt,X',Y',Z',X,Y,Z,tiempo,amp,frec_hits,t_selp
0,25800,73557,0,2,76,4,19,0,45.532615,9.219245,90.040108,0.693344,0.138831,-0.707106,296.559100,1.089169,327.587128,137.146191
1,25800,73557,0,2,77,4,19,0,45.532615,9.219245,90.040108,0.693344,0.138831,-0.707106,137.146191,0.879543,327.587128,137.146191
2,25800,73557,0,2,78,4,19,1,44.584223,9.539780,90.040108,-0.466903,0.531039,-0.707106,153.189412,0.772948,281.085968,137.146191
3,25800,73557,0,2,67,4,20,0,45.484239,9.361442,104.567108,0.634163,0.312791,-0.707106,168.470446,1.070132,82.550049,167.488297
4,25800,73557,0,2,68,4,20,0,45.484239,9.361442,104.567108,0.634163,0.312791,-0.707106,897.750991,0.985427,82.550049,167.488297


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63520301 entries, 0 to 63520300
Data columns (total 18 columns):
runID              int64
frameID            int64
trigger_counter    int64
interactionID      int64
#hit               int64
lin_det            int64
piso_det           int64
mod_opt            int64
X'                 float64
Y'                 float64
Z'                 float64
X                  float64
Y                  float64
Z                  float64
tiempo             float64
amp                float64
frec_hits          float64
t_selp             float64
dtypes: float64(10), int64(8)
memory usage: 8.5 GB


In [6]:
#Vamos a obtener la columna de t_selp que queremos para pegarla al dataframe "limpio" que ya teníamos

#Primero abrimos el df limpio

df_limpio = pd.read_csv(cleandata+"df_best_hits_limpio.csv")

In [9]:
print(df_limpio.shape)
df_limpio.head(5)

(63520301, 17)


,Unnamed: 0,runID,frameID,trigger_counter,interactionID,#hit,lin_det,piso_det,mód_opt,X',Y',Z',azimut,zenit,tiempo,amp,frec_hits
0,0,25800,73557,0,2,76,4,19,0,45.532615,9.219245,90.040108,0.197620,2.356193,296.559100,1.089169,327.587128
1,1,25800,73557,0,2,77,4,19,0,45.532615,9.219245,90.040108,0.197620,2.356193,137.146191,0.879543,327.587128
2,2,25800,73557,0,2,78,4,19,1,44.584223,9.539780,90.040108,2.292014,2.356193,153.189412,0.772948,281.085968
3,3,25800,73557,0,2,67,4,20,0,45.484239,9.361442,104.567108,0.458220,2.356193,168.470446,1.070132,82.550049
4,4,25800,73557,0,2,68,4,20,0,45.484239,9.361442,104.567108,0.458220,2.356193,897.750991,0.985427,82.550049


In [33]:
#Quitando la columna Unnamed, que de vdd no sé porque sale

df_limpio = df_limpio.drop([df_limpio.columns[0]], axis='columns')

df_limpio.shape

(63520301, 16)

In [34]:
#Selecciono la columna t_selp
columna_tselp = df[['t_selp']]

print(type(columna_tselp), columna_tselp.shape)
columna_tselp.head(6)

<class 'pandas.core.frame.DataFrame'> (63520301, 1)


,t_selp
0,137.146191
1,137.146191
2,137.146191
3,167.488297
4,167.488297
5,167.488297


In [35]:
df_limpio.head(2)

,runID,frameID,trigger_counter,interactionID,#hit,lin_det,piso_det,mód_opt,X',Y',Z',azimut,zenit,tiempo,amp,frec_hits
0,25800,73557,0,2,76,4,19,0,45.532615,9.219245,90.040108,0.19762,2.356193,296.559100,1.089169,327.587128
1,25800,73557,0,2,77,4,19,0,45.532615,9.219245,90.040108,0.19762,2.356193,137.146191,0.879543,327.587128


In [37]:
#Agrego-inserto la columna correspondiente al t_selp al dataframe limpio
#algo así como "sólo hay que cortar la columna t_selp y pegarla al otro dataframe".

df_limpio.insert(14, "t_selp", columna_tselp)

df_limpio.head(5)

,runID,frameID,trigger_counter,interactionID,#hit,lin_det,piso_det,mód_opt,X',Y',Z',azimut,zenit,tiempo,t_selp,amp,frec_hits
0,25800,73557,0,2,76,4,19,0,45.532615,9.219245,90.040108,0.197620,2.356193,296.559100,137.146191,1.089169,327.587128
1,25800,73557,0,2,77,4,19,0,45.532615,9.219245,90.040108,0.197620,2.356193,137.146191,137.146191,0.879543,327.587128
2,25800,73557,0,2,78,4,19,1,44.584223,9.539780,90.040108,2.292014,2.356193,153.189412,137.146191,0.772948,281.085968
3,25800,73557,0,2,67,4,20,0,45.484239,9.361442,104.567108,0.458220,2.356193,168.470446,167.488297,1.070132,82.550049
4,25800,73557,0,2,68,4,20,0,45.484239,9.361442,104.567108,0.458220,2.356193,897.750991,167.488297,0.985427,82.550049


In [38]:
df_limpio.sample(10)
#Cabe hacer notar que verifiqué manualmente la línea 31217621 y resultó 
#ser totalmente correcta la relación tselp vs tiempo del hit

,runID,frameID,trigger_counter,interactionID,#hit,lin_det,piso_det,mód_opt,X',Y',Z',azimut,zenit,tiempo,t_selp,amp,frec_hits
58712198,83740,17813,1192,2,201,1,15,2,4.544751,97.254659,23.341566,1.238685,2.356193,1265.554476,1265.554476,1.144843,60.310364
60591156,84480,216312,354,1,156,9,13,0,-93.738383,8.567942,3.058566,-0.209775,2.356193,-2471.219207,189.209548,0.952530,135.898590
31217621,59260,20349,199,1,49,2,19,0,76.805180,57.809734,90.170566,0.653550,2.356193,368.710795,368.710795,1.546130,182.991028
21741474,52340,47121,1730,1,83,4,14,0,44.874045,10.109058,17.402566,1.729320,2.356193,418.954574,322.440831,1.299455,141.029358
50959152,74780,156313,363,1,93,5,19,1,-57.807253,78.556881,89.978566,-0.071791,2.356193,2681.768129,454.775381,1.075822,368.328094
15543222,45780,94286,3352,1,8,4,22,2,44.402305,9.668962,133.617566,2.913604,2.356193,1652.573953,1652.573953,1.050960,60.482025
8350987,36220,6967,537,2,92,4,23,1,45.367534,9.521314,148.146108,0.802330,2.356193,-1660.850067,462.068302,1.748227,251.483917
6560235,34470,21850,1034,2,54,2,14,2,76.914388,57.135027,17.456108,0.190256,2.356193,1180.256904,1165.499665,2.356897,69.999695
52655945,77400,159110,668,1,55,2,18,1,75.803854,57.258839,75.632566,-2.789271,2.356193,1798.271695,276.240617,0.277925,85.048676
34277013,61020,3871,4623,2,279,1,20,2,4.233429,96.143569,95.957566,-1.784955,2.356193,1047.943309,1047.943309,1.397089,152.835846


In [39]:
#Guardamos el dataframe como un archivo CSV para posteriormente unirlo con el otro (data aafit, bbfit...)
#Como ya tengo la transformación azimut y zenit, solo hay que verificar que las filas son las mismas 

#outputpath = "U:\MASTER INT. ARTIFICIAL\TFM_DATA\CLEANDATA\\"
outputpath = "F:\DATA_TFM\CLEANDATA\\"
    
df_limpio.to_csv(outputpath+"data_best_hits_inc_tselp.csv", index = False)

In [3]:
path = "F:\DATA_TFM\CLEANDATA\\"
archivo = "data_best_hits_inc_tselp.csv"
file = path+archivo
df_tselp = pd.read_csv(file)

In [5]:
#Esta era una idea para usar pickle pero no jaló por falta de memoria... tuve que reiniciar el kernel
#Guardando el mismo dataframe como pickle para ver cual carga más rápido después en el modelado
import pickle

df_tselp.to_pickle(path+"data_best_hits_inc_tselp.pkl")